# BAN436 Final project

This is a final project for the NHH course BAN436. In this project we will prepare for the Ames Housing Data competition. We will analyze housing data and use predictive models to predict housing prics using simple linear regression.

## Task 1: Import and explore the data

In [26]:
# import all necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn import linear_model

Now that we have all necessary libraries for this project, let's import the data.

In [5]:
# import the data
data = pd.read_csv('AmesHousing.csv')

data

,ID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,...,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,...,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,Norm,1Fam,SLvl,6,6,1984,1984,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,TA,TA,CBlock,TA,TA,Av,GLQ,819.0,Unf,0.0,184.0,1003.0,GasA,...,Y,SBrkr,1003,0,0,1003,1.0,0.0,1,0,3,1,TA,6,Typ,0,NaN,Detchd,1984.0,Unf,2.0,588.0,TA,TA,Y,120,0,0,0,0,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,1Story,5,5,1983,1983,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,TA,TA,CBlock,Gd,TA,Av,BLQ,301.0,ALQ,324.0,239.0,864.0,GasA,...,Y,SBrkr,902,0,0,902,1.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1983.0,Unf,2.0,484.0,TA,TA,Y,164,0,0,0,0,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,NaN,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,337.0,Unf,0.0,575.0,912.0,GasA,...,Y,SBrkr,970,0,0,970,0.0,1.0,1,0,3,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,80,32,0,0,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,924100070,20,RL

There seem to be 2930 observations and 81 variables. The first variable is the house ID and last is selling price, so that leaves us with 79 potential explanatory variables. Let's get some more info about the data set.

In [11]:
# get descriptive statistics
data['SalePrice'].describe()

count      2930.000000
mean     180796.060068
std       79886.692357
min       12789.000000
25%      129500.000000
50%      160000.000000
75%      213500.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [10]:
# get info about the variables
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             2930 non-null   int64  
 1   MSSubClass     2930 non-null   int64  
 2   MSZoning       2930 non-null   object 
 3   LotFrontage    2440 non-null   float64
 4   LotArea        2930 non-null   int64  
 5   Street         2930 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2930 non-null   object 
 8   LandContour    2930 non-null   object 
 9   Utilities      2930 non-null   object 
 10  LotConfig      2930 non-null   object 
 11  LandSlope      2930 non-null   object 
 12  Neighborhood   2930 non-null   object 
 13  Condition1     2930 non-null   object 
 14  Condition2     2930 non-null   object 
 15  BldgType       2930 non-null   object 
 16  HouseStyle     2930 non-null   object 
 17  OverallQual    2930 non-null   int64  
 18  OverallC

Given the amount of variables, we focus on the sale price to keep it simple. The average price is $180,796.06. The cheapes house was sold for $12,789 and the most expensive one for $755,000. it is worth noting that there is quite a gap between these extremes and lower and upper quartiles. We can assume the presence of a few outliers on both sides.

As for the variables themselves, we notice a majority of categorical variables (43). 36 variables are numerical, some integers other floats. From the data description available we need to also consider some categorical variables are perceived as numerical. We can also already see that there are plenty of missing values in the dataset. For example 'Alley' has only 198 non-null observations out of 2930. This is not because the data is missing but because there is no alley access. Let's look at missing values.

In [68]:
# count missing values
na_values = data.isnull().sum()[data.isnull().sum() > 0]

na_values

LotFrontage      490
Alley           2732
MasVnrType      1775
MasVnrArea        23
BsmtQual          80
BsmtCond          80
BsmtExposure      83
BsmtFinType1      80
BsmtFinSF1         1
BsmtFinType2      81
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
FireplaceQu     1422
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2917
Fence           2358
MiscFeature     2824
dtype: int64

We see many variables have missing data. However, a lot of them are not actually missing, rather they cannot be measured because the feature does not exist. We can see there are approximately 80 homes without basement, 160 homes without garage and 1422 homes without a fireplace. Many are also without pools and fences. Since there is a lot of "missing" data regarding these variables, we cannot simply drop the observations. We will have to assign a value to the instead.

Since we will be predicting home prices with simple regression, there is no need to keep all variables. Let's look at the correlation between sales price and other variables. We can drop those taht have no or very weak correlation later.

In [25]:
# get correlation
data.corr(numeric_only=True)

,ID,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
ID,1.000000,-0.001281,-0.096918,0.034868,-0.263147,0.104451,-0.343388,-0.157111,-0.229283,-0.098375,-0.001145,-0.087707,-0.189642,-0.141902,-0.003289,0.056940,-0.107579,-0.037759,0.004328,-0.171431,-0.166636,0.006345,0.076470,-0.068981,-0.108056,-0.256829,-0.237484,-0.210606,-0.051135,-0.071311,0.162519,-0.024894,-0.025735,-0.002845,-0.008260,-0.050455,0.009579,-0.246521
MSSubClass,-0.001281,1.000000,-0.420135,-0.204613,0.039419,-0.067349,0.036579,0.043397,0.002730,-0.060075,-0.070946,-0.130421,-0.219445,-0.247828,0.304237,0.025765,0.068061,0.013701,-0.003329,0.134631,0.175879,-0.019208,0.257698,0.031898,-0.049955,0.088754,-0.045883,-0.103239,-0.017310,-0.014823,-0.022866,-0.037956,-0.050614,-0.003434,-0.029254,0.000350,-0.017905,-0.085092
LotFrontage,-0.096918,-0.420135,1.000000,0.491313,0.212042,-0.074448,0.121562,0.091712,0.222407,0.215583,0.045999,0.116743,0.353773,0.457391,0.029187,0.005249,0.383822,0.108915,-0.024724,0.184521,0.041880,0.240442,0.005407,0.353137,0.257255,0.076306,0.308706,0.358505,0.120084,0.163040,0.012758,0.028564,0.076666,0.173947,0.044476,0.011085,-0.007547,0.357318
LotArea,0.034868,-0.204613,0.491313,1.000000,0.097188,-0.034759,0.023258,0.021682,0.126830,0.191555,0.083150,0.023658,0.253589,0.332235,0.032996,0.000812,0.285599,0.125877,0.026903,0.127433,0.035497,0.136569,-0.020301,0.216597,0.256989,-0.008952,0.179512,0.212822,0.157212,0.103760,0.021868,0.016243,0.055044,0.093775,0.069188,0.003859,-0.023085,0.266549
OverallQual,-0.263147,0.039419,0.212042,0.097188,1.000000,-0.094812,0.597027,0.569609,0.429418,0.284118,-0.041287,0.270058,0.547294,0.477837,0.241402,-0.048680,0.570556,0.167858,-0.041647,0.522263,0.268853,0.063291,-0.159744,0.380693,0.393007,0.570569,0.599545,0.563503,0.255663,0.298412,-0.140332,0.018240,0.041615,0.030399,0.005179,0.031103,-0.020719,0.799262
OverallCond,0.104451,-0.067349,-0.074448,-0.034759,-0.094812,1.000000,-0.368773,0.047680,-0.135340,-0.050935,0.041134,-0.136819,-0.173344,-0.157052,0.006218,0.009175,-0.115643,-0.042766,0.084455,-0.214316,-0.088127,-0.006137,-0.086386,-0.089816,-0.031702,-0.326017,-0.181557,-0.153754,0.020344,-0.068934,0.071459,0.043852,0.044055,-0.016787,0.034056,-0.007295,0.031207,-0.101697
YearBuilt,-0.343388,0.036579,0.121562,0.023258,0.597027,-0.368773,1.000000,0.612095,0.313292,0.279870,-0.027415,0.128998,0.407526,0.310463,0.016828,-0.144282,0.241726,0.211849,-0.030626,0.469406,0.269268,-0.055093,-0.137852,0.111919,0.170672,0.834849,0.537443,0.480131,0.228964,0.198365,-0.374364,0.015803,-0.041436,0.002213,-0.011011,0.014577,-0.013197,0.558426
YearRemodAdd,-0.157111,0.043397,0.091712,0.021682,0.569609,0.047680,0.612095,1.000000,0.196928,0.151790,-0.062129,0.164805,0.297481,0.242108,0.158939,-0.060365,0.316855,0.134387,-0.046292,0.457266,0.211771,-0.021536,-0.142404,0.197528,0.133322,0.652310,0.425403,0.376438,0.217857,0.241748,-0.220383,0.037412,-0.046888,-0.011410,-0.003132,0.018048,0.032652,0.532974
MasVnrArea,-0.229283,0.002730,0.222407,0.126830,0.429418,-0.135340,0.313292,0.196928,1.000000,0.301872,-0.016019,0.091668,0.397040,0.395736,0.121805,-0.057701,0.403611,0.140113,0.015421,0.260153,0.192965,0.080546,-0.050998,0.279563,0.272068,0.254784,0.360159,0.373458,0.165467,0.143748,-0.110787,0.013778,0.065643,0.004617,0.044934,-0.000276,-0.017715,0.508285
BsmtFinSF1,-0.098375,-0.060075,0.215583,0.191555,0.284118,-0.050935,0.279870,0.151790,0.301872,1.000000,-0.054129,-0.477875,0.536547,0.457472,-0.164014,-0.066173,0.209633,0.640020,0.077548,0.077772,-0.008457,-0.118959,-0.086738,0.047631,0.295882,0.194238,0.255483,0.309876,0.224010,0.124947,-0.100455

From a quick analysis we can see that there are quite high correlations between sales price and different variables. The highest are for example overall quality, year of construction or renovation, area of different features. Note that these are only numeric variables. We will transform the data later and choose the variables then.

## Task 2: Visualize the data

We can also analyze tha data visually. Let's create a few plots to understand possible relationships of the data. Firs, let's look at the dependent variable 'SalePrice'.

In [28]:
# histogram
plot_1 = px.histogram(
    data,
    x='SalePrice'
)

plot_1.show()

We can see that there are noticable outliers of homes that cost more that $500k. These observations might affect results significnatly so it is worht considering droping these.

Next, let's consider the correlations. We noticed that there is a strong correlation between sales price and other variables. Scatterplots should give us a better idea what kind of relationship it is (linear or other). We will use a function so we can choose any variables anytime.

In [31]:
# scatterplot function
def print_scatterplots(data, dep, expl):
    for var in expl:
        plot = px.scatter(
            data,
            x=var,
            y=dep,
            title=f'{dep} X {var}'
        )

        plot.show()

In [32]:
print_scatterplots(data, 'SalePrice', ['OverallQual', 'YearBuilt', 'YearRemodAdd', '1stFlrSF'])

We can see possible linear relationships between sales price and other explanatory variables. We can reuse the function with other variables if necessary.

## Task 3: Data wrangling

Now, the time has come to address the flaws of the data. We need to handle the missing values, transform string data to categorical variables, remove unwanted outliers and decide on the variables we will use for predictions.

#### Missing values

There are 27 variables with missing values. We need to identify truly missing values and data unavailable for simply not existing. Let's assume that missing values in numeric variables are labeled correctly as missing. We will get rid of these observations as there should not be that many of them. This also allows us to keep the same observations for each simple regression, it will be easier to compare the results.

In [77]:
# get a copy of the data to clean
data_clean = data.copy()

# filter and delete missing values from numeric variables
data_clean = data_clean.dropna(subset=data_clean[na_values.index].select_dtypes('float').columns)

na_values_clean = data_clean.isnull().sum()[data_clean.isnull().sum() > 0]

data_clean[na_values_clean.index].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2274 entries, 0 to 2929
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Alley         162 non-null    object
 1   MasVnrType    913 non-null    object
 2   BsmtQual      2223 non-null   object
 3   BsmtCond      2223 non-null   object
 4   BsmtExposure  2220 non-null   object
 5   BsmtFinType1  2223 non-null   object
 6   BsmtFinType2  2222 non-null   object
 7   Electrical    2273 non-null   object
 8   FireplaceQu   1174 non-null   object
 9   PoolQC        12 non-null     object
 10  Fence         445 non-null    object
 11  MiscFeature   74 non-null     object
dtypes: object(12)
memory usage: 231.0+ KB


Now we have data where missing values are only for string (object) type variables. Notice that the code before also dropped 4 variables that had the same missing observations. Lucky us now have to deal with only 12 variables. After consulting the data description file, we see that except "Electrical" variable, all other are simply missing the features, not observations. Lets drop the one observation for "Electrical" and replace the NA values of other variables with "none".

In [81]:
# drop missing values for "electrical" variable
data_clean = data_clean.dropna(subset='Electrical')

# replace other missing values with "none"
data_clean = data_clean.fillna('none')

data_clean.isna().sum() # no missing values

ID               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

#### Categorical variables

In this section we will need to transform and recode some of the variables. There are four types of variables we need to deal with:
* Categorical as numeric (ordered)
* Categorical as numeric (unordered)
* Categorical as string (ordered)
* Categorical as string (unordered)

With pandas, it is easy to transform data into categorical type


In [91]:
df = data_clean.copy()

# df['MSSubClass'] = df['MSSubClass'].astype('category')
df['MSSubClass'] = pd.Categorical(df['MSSubClass'], ordered=True)

df['MSSubClass'].max()

np.int64(190)